In [ ]:
!pip install -qU xformers transformers 
!pip install -q unsloth
!git clone https://github.com/SparkAudio/Spark-TTS
!pip install -q omegaconf einx einops soundfile librosa

In [ ]:
import tqdm.notebook as tqdm
from unsloth import FastModel
from transformers import CsmForConditionalGeneration
import torch
import datasets
from IPython.display import Audio, display
from huggingface_hub import snapshot_download
from huggingface_hub import hf_hub_download
import torch
import re
import numpy as np
from typing import Dict, Any
import torchaudio.transforms as T

In [ ]:
# Get the base Spark-TTS repo
snapshot_download("unsloth/Spark-TTS-0.5B", local_dir = "Spark-TTS-0.5B")
# And overwrite the model with the one we fine-tuned for Ugandan languages
model_local_path = hf_hub_download(repo_id="jq/spark-tts-salt", filename="model.safetensors")
!mv {model_local_path} Spark-TTS-0.5B/LLM/

In [ ]:
model, tokenizer = FastModel.from_pretrained(
    model_name = f"Spark-TTS-0.5B/LLM",
    max_seq_length = 2048,
    dtype = torch.float32, 
    full_finetuning = True,
    load_in_4bit = False,
)

In [ ]:
import sys
sys.path.append('Spark-TTS')
from sparktts.models.audio_tokenizer import BiCodecTokenizer
from sparktts.utils.audio import audio_volume_normalize

audio_tokenizer = BiCodecTokenizer("Spark-TTS-0.5B", "cuda")

In [ ]:
FastModel.for_inference(model) # Enable native 2x faster inference

@torch.inference_mode()
def generate_speech_from_text(
    text: str,
    temperature: float = 0.8,   # Generation temperature
    top_k: int = 50,            # Generation top_k
    top_p: float = 1,        # Generation top_p
    max_new_audio_tokens: int = 2048, # Max tokens for audio part
    device: torch.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
) -> np.ndarray:
    """
    Generates speech audio from text using default voice control parameters.

    Args:
        text (str): The text input to be converted to speech.
        temperature (float): Sampling temperature for generation.
        top_k (int): Top-k sampling parameter.
        top_p (float): Top-p (nucleus) sampling parameter.
        max_new_audio_tokens (int): Max number of new tokens to generate (limits audio length).
        device (torch.device): Device to run inference on.

    Returns:
        np.ndarray: Generated waveform as a NumPy array.
    """

    prompt = "".join([
        "<|task_tts|>",
        "<|start_content|>",
        text,
        "<|end_content|>",
        "<|start_global_token|>"
    ])
    
    model_inputs = tokenizer([prompt], return_tensors="pt").to(device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_audio_tokens, # Limit generation length
        do_sample=True,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        eos_token_id=tokenizer.eos_token_id, # Stop token
        pad_token_id=tokenizer.pad_token_id # Use models pad token id
    )

    generated_ids_trimmed = generated_ids[:, model_inputs.input_ids.shape[1]:]

    predicts_text = tokenizer.batch_decode(generated_ids_trimmed, skip_special_tokens=False)[0]

    # Extract semantic token IDs using regex
    semantic_matches = re.findall(r"<\|bicodec_semantic_(\d+)\|>", predicts_text)
    if not semantic_matches:
        print("Warning: No semantic tokens found in the generated output.")
        # Handle appropriately - perhaps return silence or raise error
        return np.array([], dtype=np.float32)

    pred_semantic_ids = torch.tensor([int(token) for token in semantic_matches]).long().unsqueeze(0) # Add batch dim

    # Extract global token IDs using regex (assuming controllable mode also generates these)
    global_matches = re.findall(r"<\|bicodec_global_(\d+)\|>", predicts_text)
    if not global_matches:
         print("Warning: No global tokens found in the generated output (controllable mode). Might use defaults or fail.")
         pred_global_ids = torch.zeros((1, 1), dtype=torch.long)
    else:
         pred_global_ids = torch.tensor([int(token) for token in global_matches]).long().unsqueeze(0) # Add batch dim

    pred_global_ids = pred_global_ids.unsqueeze(0) # Shape becomes (1, 1, N_global)

    return pred_global_ids, pred_semantic_ids

In [ ]:
# Speaker IDs:
# 241: Acholi (female)
# 242: Ateso (female)
# 243: Runyankore (female)
# 245: Lugbara (female)
# 246: Swahili (male)
# 248: Luganda (female)

In [ ]:
input_text = "248: I am a language assistant made by Sunbird AI in Uganda."

pred_global_ids, pred_semantic_ids = generate_speech_from_text(input_text)
wav_np = audio_tokenizer.detokenize(
    pred_global_ids.to('cuda').squeeze(0),
    pred_semantic_ids.to('cuda') 
)
display(Audio(wav_np, rate=16_000))

In [ ]:
# Slow down or speed up by changing the sample rate
display(Audio(wav_np, rate=14_000))